In [ ]:
#!pip install cufflinks
#!pip install plotly

In [ ]:
# Imports
from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
import pandas_profiling
import datetime

In [ ]:
# Plot settings
import plotly
import cufflinks
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, display_html

# set formatting
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [ ]:
# Import stock file
df_raw = pd.read_csv('GE.csv', index_col = 0, parse_dates=True)

# Look at data head
df_raw.head()

In [ ]:
# look at data tail
df_raw.tail()

In [ ]:
# look a shape of data
df_raw.shape

In [ ]:
# Import file
pandas_profiling.ProfileReport(df_raw)

In [ ]:
# look at data types
df_raw.dtypes

In [ ]:
df_temp = df_raw

In [ ]:
df = df_temp

# Should it stay or should it go?

In [ ]:
# Group data by date
df = df_temp.groupby(by = 'Date').agg('Adj Close')

In [ ]:
# Change index to datetime
df.index = pd.to_datetime(df_temp.index)

In [ ]:
# Set frequency of time series
df = df_temp.asfreq(freq='1D')

In [ ]:
# Show the end of the data
df.tail()

In [ ]:
display(df)

In [ ]:
# Drop unused columns
df = df.drop(['Close'], axis=1)
df = df.drop(['Open'], axis=1)
df = df.drop(['High'], axis=1)
df = df.drop(['Low'], axis=1)
df = df.drop(['Volume'], axis=1)

In [ ]:
# Rename 'Adj Close' column
df = df.rename(columns = {'Adj Close' : 'ts'})

# Plot

In [ ]:
# Plot time series data
f, ax = plt.subplots(1,1)
ax.plot(df_raw['Adj Close'])

# Add title
ax.set_title('Time-series graph for 1 time-series example')

# Rotate x-labels
ax.tick_params(axis = 'x', rotation = 45)

# Show graph
plt.show()
#plt.close()

In [ ]:
    ## Plot of Raw Adj Closing prices
## df['Close'].plot()

In [ ]:
# Plot of Adj Closing prices
df['ts'].plot()

In [ ]:
# Impute and plot time series with interpolated missing values (non trading days)
df = df.assign(InterpolateTime=df['ts'].interpolate(method='time'))
df.plot()

In [ ]:
df['ts'] = InterpolateTime=df['ts'].interpolate(method='time')

In [ ]:
# Plot of Adj Closing prices
df['ts'].plot()

In [ ]:
df.diff().head()
# df['Return'] = df.Close - df.Open

In [ ]:
# Compute price changes from one day to the other
df_diff = df.diff()

In [ ]:
# Plot of price change series from one day to the other (Adj Close)
df_diff['ts'].plot()

In [ ]:
    ## Plot of price change series from one day to the other (Close)
## df_diff['Close'].plot()

In [ ]:
# Log transormation
df_transformed= np.log(df['ts'] / df['ts'].shift(1))


In [ ]:
#PLot log transofrmation series
df_transformed.plot()

In [ ]:
date = df.index
adj_close = df['ts']

In [ ]:
adj_close.plot()

# **Applying statistical modeling and machine learning to perform time-series forecasting**

### Look at Stationarity

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(df, ts):
    """
    Test stationarity using moving average statistics and Dickey-Fuller test
    Source: https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/
    """
    
    # Determing rolling statistics
    rolmean = df[ts].rolling(window = 12, center = False).mean()
    rolstd = df[ts].rolling(window = 12, center = False).std()
    
    # Plot rolling statistics:
    orig = plt.plot(df[ts], 
                    color = 'blue', 
                    label = 'Original')
    mean = plt.plot(rolmean, 
                    color = 'red', 
                    label = 'Rolling Mean')
    std = plt.plot(rolstd, 
                   color = 'black', 
                   label = 'Rolling Std')
    plt.legend(loc = 'best')
    plt.title('Rolling Mean & Standard Deviation for %s' %(ts))
    plt.xticks(rotation = 45)
    plt.show(block = False)
    plt.close()
    
    # Perform Dickey-Fuller test:
    # Null Hypothesis (H_0): time series is not stationary
    # Alternate Hypothesis (H_1): time series is stationary
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(df[ts], 
                      autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], 
                         index = ['Test Statistic',
                                  'p-value',
                                  '# Lags Used',
                                  'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
test_stationarity(df = df, ts = 'ts')

### Transforming, Smoothing, Differencing

In [ ]:
# The next two cells create a dataframe that apply a lot of transformations to the raw series.
# Some combinations of transformations are also tested.
# This allows to choose the transformation/s that make the data more stationary over time.
# Transformations make the time series stationary based on the Dickey-Fuller test when
# the p-value = < 0.05.


def plot_transformed_data(df, ts, ts_transform):
    """
    Plot transformed and original time series data
    """
    # Plot time series data
    f, ax = plt.subplots(1,1)
    ax.plot(df[ts])
    ax.plot(df[ts_transform], color = 'red')

    # Add title
    ax.set_title('%s and %s time-series graph' %(ts, ts_transform))

    # Rotate x-labels
    ax.tick_params(axis = 'x', rotation = 45)

    # Add legend
    ax.legend([ts, ts_transform])

    plt.show()
    plt.close()

    return

In [ ]:
# Transformation - log ts
df['ts_log'] = df['ts'].apply(lambda x: np.log(x))

# Transformation - 7-day moving averages of log ts
df['ts_log_moving_avg'] = df['ts_log'].rolling(window = 7,
                                                               center = False).mean()

# Transformation - 7-day moving average ts
df['ts_moving_avg'] = df['ts'].rolling(window = 7,
                                                       center = False).mean()

# Transformation - Difference between logged ts and first-order difference logged ts
# df['ts_log_diff'] = df['ts_log'] - df['ts_log'].shift()
df['ts_log_diff'] = df['ts_log'].diff()

# Transformation - Difference between ts and moving average ts
df['ts_moving_avg_diff'] = df['ts'] - df['ts_moving_avg']

# Transformation - Difference between logged ts and logged moving average ts
df['ts_log_moving_avg_diff'] = df['ts_log'] - df['ts_log_moving_avg']

# Transformation - Difference between logged ts and logged moving average ts
df_transform = df.dropna()

# Transformation - Logged exponentially weighted moving averages (EWMA) ts
df_transform['ts_log_ewma'] = df_transform['ts_log'].ewm(halflife = 7,
                                                                         ignore_na = False,
                                                                         min_periods = 0,
                                                                         adjust = True).mean()

# Transformation - Difference between logged ts and logged EWMA ts
df_transform['ts_log_ewma_diff'] = df_transform['ts_log'] - df_transform['ts_log_ewma']

# Display data
display(df_transform.head())

# Plot data
plot_transformed_data(df = df, 
                      ts = 'ts', 
                      ts_transform = 'ts_log')
# Plot data
plot_transformed_data(df = df, 
                      ts = 'ts_log', 
                      ts_transform = 'ts_log_moving_avg')

# Plot data
plot_transformed_data(df = df_transform, 
                      ts = 'ts', 
                      ts_transform = 'ts_moving_avg')

# Plot data
plot_transformed_data(df = df_transform, 
                      ts = 'ts_log', 
                      ts_transform = 'ts_log_diff')

# Plot data
plot_transformed_data(df = df_transform, 
                      ts = 'ts', 
                      ts_transform = 'ts_moving_avg_diff')

# Plot data
plot_transformed_data(df = df_transform, 
                      ts = 'ts_log', 
                      ts_transform = 'ts_log_moving_avg_diff')

# Plot data
plot_transformed_data(df = df_transform, 
                      ts = 'ts_log', 
                      ts_transform = 'ts_log_ewma')

# Plot data
plot_transformed_data(df = df_transform, 
                      ts = 'ts_log', 
                      ts_transform = 'ts_log_ewma_diff')

# Perform stationarity test
test_stationarity(df = df_transform, 
                  ts = 'ts_log')

# Perform stationarity test
test_stationarity(df = df_transform, 
                  ts = 'ts_moving_avg')

# Perform stationarity test
test_stationarity(df = df_transform, 
                  ts = 'ts_log_moving_avg')

# Perform stationarity test
test_stationarity(df = df_transform,
                  ts = 'ts_log_diff')

# Perform stationarity test
test_stationarity(df = df_transform,
                  ts = 'ts_moving_avg_diff')

# Perform stationarity test
test_stationarity(df = df_transform,
                  ts = 'ts_log_moving_avg_diff')

# Perform stationarity test
test_stationarity(df = df_transform, 
                  ts = 'ts_log_ewma')

# Perform stationarity test
test_stationarity(df = df_transform,
                  ts = 'ts_log_ewma_diff')

### **ARIMA models**

In [ ]:
def plot_acf_pacf(df, ts):
    """ Plot auto-correlation function (ACF) and partial auto-correlation (PACF) plots """
    f, (ax1, ax2) = plt.subplots(1,2, figsize = (10, 5))
    
    #Plot ACF: 

    ax1.plot(lag_acf)
    ax1.axhline(y=0,linestyle='--',color='gray')
    ax1.axhline(y=-1.96/np.sqrt(len(df[ts])),linestyle='--',color='gray')
    ax1.axhline(y=1.96/np.sqrt(len(df[ts])),linestyle='--',color='gray')
    ax1.set_title('Autocorrelation Function for %s' %(ts))

    #Plot PACF:
    ax2.plot(lag_pacf)
    ax2.axhline(y=0,linestyle='--',color='gray')
    ax2.axhline(y=-1.96/np.sqrt(len(df[ts])),linestyle='--',color='gray')
    ax2.axhline(y=1.96/np.sqrt(len(df[ts])),linestyle='--',color='gray')
    ax2.set_title('Partial Autocorrelation Function for %s' %(ts))

    plt.tight_layout()
    plt.show()
    plt.close()
    
    return

In [ ]:
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf

# determine ACF and PACF
lag_acf = acf(np.array(df_transform['ts_log_diff']), nlags = 20)
lag_pacf = pacf(np.array(df_transform['ts_log_diff']), nlags = 20)

# plot ACF and PACF
plot_acf_pacf(df = df_transform, ts = 'ts_log_diff')

In [ ]:
def run_arima_model(df, ts, p, d, q):
    """
    Run ARIMA model
    """
    from statsmodels.tsa.arima_model import ARIMA

    # fit ARIMA model on time series
    model = ARIMA(df[ts], order=(p, d, q))  
    results_ = model.fit(disp=-1)  

    # get lengths correct to calculate RSS
    len_results = len(results_.fittedvalues)
    ts_modified = df[ts][-len_results:]

    # calculate root mean square error (RMSE) and residual sum of squares (RSS)
    rss = sum((results_.fittedvalues - ts_modified)**2)
    rmse = np.sqrt(rss / len(df[ts]))

    # plot fit
    plt.plot(df[ts])
    plt.plot(results_.fittedvalues, color = 'red')
    plt.title('For ARIMA model (%i, %i, %i) for ts %s, RSS: %.4f, RMSE: %.4f' %(p, d, q, ts, rss, rmse))

    plt.show()
    plt.close()

    return results_

In [ ]:
# Note: I do the differencing in the transformation of the data 'ts_log_diff'
# AR model with 1st order differencing - ARIMA (1,0,0)


model_AR = run_arima_model(df = df_transform, 
                           ts = 'ts_log_diff', 
                           p = 1, 
                           d = 0, 
                           q = 0)

# MA model with 1st order differencing - ARIMA (0,0,1)
model_MA = run_arima_model(df = df_transform, 
                           ts = 'ts_log_diff', 
                           p = 0, 
                           d = 0, 
                           q = 1)

# ARMA model with 1st order differencing - ARIMA (1,0,1)
model_MA = run_arima_model(df = df_transform, 
                           ts = 'ts_log_diff', 
                           p = 1, 
                           d = 0, 
                           q = 1)

In [ ]:
!pip install pystan
!pip install fbprophet
from fbprophet import Prophet
import datetime
from datetime import datetime

In [ ]:
def days_between(d1, d2):
    """Calculate the number of days between two dates.  D1 is start date (inclusive) and d2 is end date (inclusive)"""
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days + 1)

In [ ]:
# Inputs for query

date_column = 'dt'
metric_column = 'ts'
table = df
start_training_date = '2010-07-03'
end_training_date = '2018-09-08'
start_forecasting_date = '2018-09-09'
end_forecasting_date = '2018-12-31'
year_to_estimate = '2018'

# Inputs for forecasting

# future_num_points
# If doing different time intervals, change future_num_points
future_num_points = days_between(start_forecasting_date, end_forecasting_date)

cap = None # 2e6

# growth: default = 'linear'
# Can also choose 'logistic'
growth = 'linear'

# n_changepoints: default = 25, uniformly placed in first 80% of time series
n_changepoints = 25 

# changepoint_prior_scale: default = 0.05
# Increasing it will make the trend more flexible
changepoint_prior_scale = 0.05 

# changpoints: example = ['2016-01-01']
changepoints = None 

# holidays_prior_scale: default = 10
# If you find that the holidays are overfitting, you can adjust their prior scale to smooth them
holidays_prior_scale = 10 

# interval_width: default = 0.8
interval_width = 0.8 

# mcmc_samples: default = 0
# By default Prophet will only return uncertainty in the trend and observation noise.
# To get uncertainty in seasonality, you must do full Bayesian sampling. 
# Replaces typical MAP estimation with MCMC sampling, and takes MUCH LONGER - e.g., 10 minutes instead of 10 seconds.
# If you do full sampling, then you will see the uncertainty in seasonal components when you plot:
mcmc_samples = 0

# holiday: default = None
# thanksgiving = pd.DataFrame({
#   'holiday': 'thanksgiving',
#   'ds': pd.to_datetime(['2014-11-27', '2015-11-26',
#                         '2016-11-24', '2017-11-23']),
#   'lower_window': 0,
#   'upper_window': 4,
# })
# christmas = pd.DataFrame({
#   'holiday': 'christmas',
#   'ds': pd.to_datetime(['2014-12-25', '2015-12-25', 
#                         '2016-12-25','2017-12-25']),
#   'lower_window': -1,
#   'upper_window': 0,
# })
# holidays = pd.concat((thanksgiving,christmas))
holidays = None

daily_seasonality = True

In [ ]:
# get relevant data - note: could also try this with ts_log_diff
df_prophet = df_transform[['ts']] # can try with ts_log_diff

# reset index
df_prophet = df_prophet.reset_index()

# rename columns
df_prophet = df_prophet.rename(columns = {'ds': 'ds', 'ts': 'y'}) # can try with ts_log_diff

# Change 'ds' type from datetime to date (necessary for FB Prophet)
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Change 'y' type to numeric (necessary for FB Prophet)
df_prophet['y'] = pd.to_numeric(df_prophet['y'], errors='ignore')

# Remove any outliers
# df.loc[(df_['ds'] > '2016-12-13') & (df_['ds'] < '2016-12-19'), 'y'] = None

In [ ]:
def create_daily_forecast(df,
                          cap,
                          holidays,
                          growth,
                          n_changepoints = 25,
                          changepoint_prior_scale = 0.05,
                          changepoints = None,
                          holidays_prior_scale = 10,
                          interval_width = 0.8,
                          mcmc_samples = 1,
                          future_num_points = 10, 
                          daily_seasonality = True):
    """
    Create forecast
    """

    # Create copy of dataframe
    df_ = df.copy()

    # Add in growth parameter, which can change over time
    #     df_['cap'] = max(df_['y']) if cap is None else cap

    # Create model object and fit to dataframe
    m = Prophet(growth = growth,
              n_changepoints = n_changepoints,
              changepoint_prior_scale = changepoint_prior_scale,
              changepoints = changepoints,
              holidays = holidays,
              holidays_prior_scale = holidays_prior_scale,
              interval_width = interval_width,
              mcmc_samples = mcmc_samples, 
              daily_seasonality = daily_seasonality)

    # Fit model with dataframe
    m.fit(df_)

    # Create dataframe for predictions
    future = m.make_future_dataframe(periods = future_num_points)
    #     future['cap'] = max(df_['y']) if cap is None else cap

    # Create predictions
    fcst = m.predict(future)

    # Plot
    m.plot(fcst);
    m.plot_components(fcst)

    return fcst

In [ ]:
fcst = create_daily_forecast(df_prophet,
                             cap,
                             holidays,
                             growth,
                             n_changepoints,
                             changepoint_prior_scale,
                             changepoints, 
                             holidays_prior_scale,
                             interval_width,
                             mcmc_samples,
                             future_num_points, 
                             daily_seasonality)

In [ ]:
def calculate_mape(y_true, y_pred):
    """ Calculate mean absolute percentage error (MAPE)"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_mpe(y_true, y_pred):
    """ Calculate mean percentage error (MPE)"""
    return np.mean((y_true - y_pred) / y_true) * 100

def calculate_mae(y_true, y_pred):
    """ Calculate mean absolute error (MAE)"""
    return np.mean(np.abs(y_true - y_pred)) * 100

def calculate_rmse(y_true, y_pred):
    """ Calculate root mean square error (RMSE)"""
    return np.sqrt(np.mean((y_true - y_pred)**2))

def print_error_metrics(y_true, y_pred):
    print('MAPE: %f'%calculate_mape(y_true, y_pred))
    print('MPE: %f'%calculate_mpe(y_true, y_pred))
    print('MAE: %f'%calculate_mae(y_true, y_pred))
    print('RMSE: %f'%calculate_rmse(y_true, y_pred))
    return

In [ ]:
print_error_metrics(y_true = df_prophet['y'], y_pred = fcst['yhat'])